In [46]:
import defs as d
import pandas as pd
import numpy as np
from fbprophet import Prophet
import datetime as dt
d

<module 'defs' from '/home/jovyan/dev/projects/vinx/defs.py'>

In [47]:
%%time
print('start')
data = pd.read_pickle(d.TMP+'/cleaned_data.pkl')
data

start
CPU times: user 18.3 ms, sys: 14.6 ms, total: 32.9 ms
Wall time: 104 ms


,store_cd,date,customer_count,post_cd,maxtemp,mintemp,rain
91,113,2017-01-27,3087,673-0041,12.7,3.1,0.0
92,113,2017-01-28,2309,673-0041,10.8,4.5,0.0
93,113,2017-01-29,2174,673-0041,10.2,3.8,3.5
94,113,2017-01-30,3248,673-0041,14.3,4.5,1.0
95,113,2017-01-31,3003,673-0041,7.1,2.2,0.0
...,...,...,...,...,...,...,...
210953,881,2020-05-07,2398,579-8051,24.1,13.5,0.0
210954,881,2020-05-08,2476,579-8051,23.2,12.5,0.0
210955,881,2020-05-09,2219,579-8051,22.9,15.6,1.5
210956,881,2020-05-10,1984,579-8051,26.2,17.3,4.0


In [48]:
%%time
print('start')

# Creates the following dictionary:
# {1: 'feature_name_1', ..., 30(horizon): 'feature_name_30'}
def rename_list(feature, horizon):
  new_col_name = []
  for ith_day in np.arange(1, horizon+1):
    str_ith_day = str(ith_day)
    new_col_name.append(feature+'_'+str_ith_day)
  rename_list = dict(zip(np.arange(1, horizon+1), new_col_name))

  return rename_list

start
CPU times: user 123 µs, sys: 0 ns, total: 123 µs
Wall time: 133 µs


In [49]:
%%time
print('start')

def feature_row_vector(date, forcast, horizon):
  output_list = []
  feature_list = forcast.index
  for feature in feature_list:

    # Select one feature
    feature_df = forcast.loc[feature]

    # Get rename_list
    cols = rename_list(feature, horizon)

    # Create a 1-row vector
    vector = pd.DataFrame(feature_df).transpose().rename(columns=cols)

    # Set date as index
    # vector.index = [date]
    # Make date as column (from index)
    # vector.reset_index(inplace=True)
    # vector.rename(columns={'index': 'date'}, inplace=True)
    # Append
    output_list.append(vector)

  output = pd.concat(output_list, axis=1)
  return output

start
CPU times: user 101 µs, sys: 8 µs, total: 109 µs
Wall time: 116 µs


In [50]:
%%time
print('start')
def forcast_features(store_cd, data, horizon):
  output_list = []

  # Create a date_list of year 2019
  mask = ('2019-01-01' <= data['date']) & (data['date'] <= '2019-12-31')
  date_list = data.loc[mask, 'date'].drop_duplicates()
  date_list_ = [date_list.iloc[0]]

  for date in date_list_:
    print('  └─ date: {}'.format(date))
    # Create training data
    mask = (data['date'] < date)
    use_cols = ['date', 'customer_count']
    train = data.loc[mask, use_cols].rename(columns={'date': 'ds', 'customer_count': 'y'})

    # We need enough data for prophet to learn
    # if False:
    if train.shape[0] >= 30:
      # Fit model
      m = Prophet()
      m.fit(train)

      # Predict
      future = m.make_future_dataframe(periods=horizon)
      forecast = m.predict(future)

      # Too many columns... Only ues yhat
      # use_cols = ['yhat', 'yhat_lower', 'yhat_upper',
      #             'trend', 'trend_lower', 'trend_upper',
      #             'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
      #             'weekly', 'weekly_lower', 'weekly_upper'
      #             ]

      # Drop unnecessary dates and columns
      mask = (date <= forecast['ds'])
      use_cols = ['yhat']
      final_forecast = forecast.loc[mask, use_cols].reset_index(drop=True).copy()

      # Make index from 1
      final_forecast.index += 1

      # Transpose !
      forcast_data = final_forecast.transpose()

      # Get a row vector representation of forcast and add to the list
      vector = feature_row_vector(date, forcast_data, horizon)
      vector['date'] = date
      output_list.append(vector)
    else:
      # Too many columns...
      # use_cols = ['yhat', 'yhat_lower', 'yhat_upper',
      #             'trend', 'trend_lower', 'trend_upper',
      #             'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
      #             'weekly', 'weekly_lower', 'weekly_upper'
      #             ]
      use_cols = ['yhat']

      # If there is not enough data, create dummy dataframe
      final_forecast = pd.DataFrame(None, index=np.arange(30), columns=use_cols)

      # Make index from 1
      final_forecast.index += 1
      # Transpose !
      forcast_data = final_forecast.transpose()

      # Get a row vector representation of dummy_df and add to the list
      vector = feature_row_vector(date, forcast_data, horizon)
      vector['date'] = date
      output_list.append(vector)

  # Concatenate row vector rep. for all dates
  output = pd.concat(output_list, axis=0)

  # Add store_cd column
  output['store_cd'] = store_cd
  return output

start
CPU times: user 116 µs, sys: 0 ns, total: 116 µs
Wall time: 127 µs


In [51]:
%%time
print('start')

# store_cd数 × 365日に対して、prophetのoutputをtransposeする
def prophet_features(data, horizon):
  output_list = []

  store_cd_list = data['store_cd'].drop_duplicates()
  store_cd_list_ = [113, 156, 174, 199, 878] # Only for short run testing

  for store_cd in store_cd_list:
    print('store_cd: {}'.format(store_cd))

    # Create store_data and if there is no record in 2019, break
    mask = (data['store_cd'] == store_cd)
    store_data = data[mask]
    loop_check = storedata['date'].dt.strftime('%Y-%m-%d').min()

    if loop_check > '2019-12-31':
      break

    # For this store_cd, get prophet forcast features for 365 days
    output_list.append(forcast_features(store_cd, store_data, horizon))

  output = pd.concat(output_list, axis=0)
  return output

start
CPU times: user 177 µs, sys: 0 ns, total: 177 µs
Wall time: 204 µs


In [52]:
%%time
print('start')
data['store_cd'].drop_duplicates()

start
CPU times: user 3.09 ms, sys: 3 ms, total: 6.09 ms
Wall time: 4.89 ms


91        113
1292      156
2493      174
3694      199
4895      211
         ... 
209841    877
210224    878
210551    879
210794    880
210864    881
Name: store_cd, Length: 193, dtype: int64

In [53]:
%%time
print('start')

hoge = prophet_features(data, 30)
hoge.to_csv(d.TMP+'/prophet_output.csv', index=False)
# hoge


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/opt/conda/envs/azureml/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling 

start
store_cd: 113
  └─ date: 2019-01-01 00:00:00
store_cd: 156
  └─ date: 2019-01-01 00:00:00
store_cd: 174
  └─ date: 2019-01-01 00:00:00
store_cd: 199
  └─ date: 2019-01-01 00:00:00
store_cd: 211
  └─ date: 2019-01-01 00:00:00
store_cd: 227
  └─ date: 2019-01-01 00:00:00
store_cd: 228
  └─ date: 2019-01-01 00:00:00
store_cd: 230
  └─ date: 2019-01-01 00:00:00
store_cd: 239
  └─ date: 2019-01-01 00:00:00
store_cd: 240
  └─ date: 2019-01-01 00:00:00
store_cd: 255
  └─ date: 2019-01-01 00:00:00
store_cd: 257
  └─ date: 2019-01-01 00:00:00
store_cd: 268
  └─ date: 2019-01-01 00:00:00
store_cd: 279
  └─ date: 2019-01-01 00:00:00
store_cd: 289
  └─ date: 2019-01-01 00:00:00
store_cd: 292
  └─ date: 2019-01-01 00:00:00
store_cd: 294
  └─ date: 2019-01-01 00:00:00
store_cd: 309
  └─ date: 2019-01-01 00:00:00
store_cd: 310
  └─ date: 2019-01-01 00:00:00
store_cd: 311
  └─ date: 2019-01-01 00:00:00
store_cd: 313
  └─ date: 2019-01-01 00:00:00
store_cd: 317
  └─ date: 2019-01-01 00:00:00
stor